# Compare the distribution of future traces as extracted by using an LSH structure to store them, with the real one 

In [1]:
from datasketch import MinHash
from sklearn import metrics
import string
import random
import difflib
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

## Helper functions

In [2]:
def random_sequence_generator(size=6, chars='abc'):
    if isinstance(size, list):
        size = random.choice(size)
    return ''.join(random.choice(chars) for _ in range(size))

In [3]:
def cosine_sim(vec1, vec2):
    return np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

In [4]:
def jaccard_similarity(list1, list2):
    list1 = list(list1)
    list2 = list(list2)
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

## LSH structure

In [5]:
class HashTable:
    def __init__(self, hash_size, inp_dimensions):
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.hash_table = dict()
        self.projections = np.random.randn(self.hash_size, inp_dimensions)
        
    def generate_hash(self, inp_vector):
        bools = (np.dot(inp_vector, self.projections.T) > 0).astype('int')
        return ''.join(bools.astype('str'))

    def __setitem__(self, inp_vec, label):
        hash_value = self.generate_hash(inp_vec)
        self.hash_table[hash_value] = self.hash_table\
            .get(hash_value, list()) + [label]
        
    def __getitem__(self, inp_vec):
        hash_value = self.generate_hash(inp_vec)
        return self.hash_table.get(hash_value, [])
        

In [6]:
class LSH:
    def __init__(self, num_tables, hash_size, inp_dimensions):
        self.num_tables = num_tables
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.hash_tables = list()
        for i in range(self.num_tables):
            self.hash_tables.append(HashTable(self.hash_size, self.inp_dimensions))
    
    def __setitem__(self, inp_vec, label):
        for table in self.hash_tables:
            table[inp_vec] = label
    
    def __getitem__(self, inp_vec):
        results = list()
        for table in self.hash_tables:
            results.extend(table[inp_vec])
        return list(set(results))


## Generate random future sequences

In [7]:
futures = [random_sequence_generator(4, 'abcd') for _ in range(1000)]

In [8]:
print('Number of different sequences: ', len(set(futures)))

Number of different sequences:  250


## Example with one hash table

In [9]:
lsh = LSH(num_tables=1, hash_size=40, inp_dimensions=10)

## Insert sequences into LSH structure

In [10]:
# Create MinHash objects

for d in futures:
    m = MinHash(num_perm=10)
    m.update(d.encode('utf8'))
    lsh.__setitem__(m.digest(),d)

In [11]:
lsh.hash_tables[0].hash_table

{'1010100101111110100001000100011000100010': ['cadb', 'cadb'],
 '1011000101111110100001000100011010100010': ['aabc',
  'aabc',
  'aabc',
  'aabc',
  'aabc',
  'aabc'],
 '0011100101111110100101010110011000000010': ['ddaa', 'ddaa', 'ddaa', 'ddaa'],
 '1011100101111110100101010110011000000010': ['bdcc',
  'cccc',
  'bdcc',
  'cddc',
  'bdcc',
  'bdcc',
  'bdcc',
  'cccc',
  'cccc',
  'cccc',
  'bdcc'],
 '0011100101011111100101011110011000000010': ['baad', 'baad', 'baad'],
 '1110100101111110100111111010111101100110': ['dcac',
  'dcac',
  'dcac',
  'dcac',
  'dcac',
  'dcac',
  'dcac',
  'dcac',
  'dcac',
  'dcac'],
 '1011100101111111100101000110011100000010': ['ccbc',
  'ccbc',
  'ccbc',
  'ccbc',
  'ccbc',
  'ccbc',
  'ccbc'],
 '1010100101111110100101000110011100100010': ['baaa', 'baaa'],
 '1010100111111110100101110110011001100010': ['caca', 'caca', 'caca', 'caca'],
 '1010000101111110100001010110011100100010': ['cabd',
  'cabd',
  'cabd',
  'cabd',
  'cabd'],
 '1010110101111110101101010110

In [12]:
print('Number of bins in the hash table:', len([1 for _ in lsh.hash_tables[0].hash_table]))

Number of bins in the hash table: 236


## Get the two distributions, the real one and the one from the LSH structure

In [13]:
ht = lsh.hash_tables[0].hash_table
f_counter = []
lsh_counter= []
for f in futures:
    f_counter += [futures.count(f)]
    for k,v in ht.items():
        if f in v:
            lsh_counter += [len(v)]
            break

f_counter = list(map(lambda x:x/sum(f_counter), f_counter))
lsh_counter = list(map(lambda x:x/sum(lsh_counter), lsh_counter))


In [14]:
def KL(a, b):
    a = np.asarray(a, dtype=np.float)
    b = np.asarray(b, dtype=np.float)

    return np.sum(np.where(a != 0, a * np.log(a / b), 0))


f_counter = np.asarray(f_counter)
lsh_counter = np.asarray(lsh_counter)

print('KL divergence: ', KL(f_counter, lsh_counter))

KL divergence:  0.0191954638258


## Example with 100 hash tables

In [15]:
lsh = LSH(num_tables=100, hash_size=40, inp_dimensions=10)

## Insert sequences into LSH structure

In [16]:
# Create MinHash objects
for d in futures:
    m = MinHash(num_perm=10)
    m.update(d.encode('utf8'))
    lsh.__setitem__(m.digest(),d)

## Get the two distributions, the real one and the one from the LSH structure

In [17]:
hts = lsh.hash_tables

f_counter = []
lsh_counter= []
for f in futures:
    f_counter += [futures.count(f)]
    lsh_counter_sum = 0
    for ht in hts:
        for k,v in ht.hash_table.items():
            if f in v:
                lsh_counter_sum += len(v)
                break
    lsh_counter += [lsh_counter_sum/len(hts)]


f_counter = list(map(lambda x:x/sum(f_counter), f_counter))
lsh_counter = list(map(lambda x:x/sum(lsh_counter), lsh_counter))


In [18]:
f_counter = np.asarray(f_counter)
lsh_counter = np.asarray(lsh_counter)

print('KL divergence: ', KL(f_counter, lsh_counter))

KL divergence:  0.00323620327535
